## ToCupy

This notebook is used to showcase the ``ToCupy`` in a variety of scenarios, and validate the changes introduced in the stacking operators and solvers to work with mixed CPU/GPU scenarios (e.g. model on GPU and data on CPU) 

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

import numpy as np
import cupy as cp
import matplotlib.pyplot as plt
import scipy as sp

from scipy.linalg import block_diag
from pylops.utils                      import dottest
from pylops.utils.wavelets             import *
from pylops.utils.seismicevents        import *
from pylops.utils.tapers               import *
from pylops.basicoperators             import *

from pylops.optimization.basic import cg as pylops_cg
from pylops.optimization.basic import cgls as pylops_cgls
from pylops.optimization.basic import lsqr as pylops_lsqr

dtype = np.float32

## VStack

In [2]:
nops = 5 # number of operators
n, m = 4, 5

# Create operator
Ms = [np.random.normal(0, 1, (n, m)) for _ in range(nops)]
M = np.vstack(Ms)
Mop = MatrixMult(M, dtype=dtype)

# Create data
x = np.ones(m, dtype=dtype)
y = Mop @ x

# Invert
xinv = Mop / y
print(xinv)

[1. 1. 1. 1. 1.]


### VStack with numpy input and numpy output

In [3]:
Mops = []
for iop in range(nops):
    Mop = MatrixMult(cp.asarray(Ms[iop]), dtype=dtype)
    Top = ToCupy(Mop.dims, dtype=dtype)
    Top1 = ToCupy(Mop.dimsd, dtype=dtype)
    Mop = Top1.H @ Mop @ Top
    Mops.append(Mop)
Mops = VStack(Mops)

In [4]:
y = Mops * x.ravel()

xinv = pylops_cgls(Mops, y)[0]
print(xinv, type(y), type(xinv))

xinv = pylops_lsqr(Mops, y)[0]
print(xinv, type(y), type(xinv))

[0.9999999 1.        1.        1.        1.       ] <class 'numpy.ndarray'> <class 'numpy.ndarray'>
[1.00000002 1.         0.99999991 1.         1.        ] <class 'numpy.ndarray'> <class 'numpy.ndarray'>


### VStack with cupy input and numpy output

In [5]:
Mops = []
for iop in range(nops):
    Mop = MatrixMult(cp.asarray(Ms[iop]), dtype=dtype)
    Top1 = ToCupy(Mop.dimsd, dtype=dtype)
    Mop = Top1.H @ Mop
    Mops.append(Mop)
Mops = VStack(Mops, inoutengine=("numpy", "cupy"))

In [6]:
xcp = cp.asarray(x)
y = Mops * xcp.ravel()

xinv = pylops_cgls(Mops, y, x0=cp.zeros_like(xcp))[0]
print(xinv, type(y), type(xinv))

xinv = pylops_lsqr(Mops, y, x0=cp.zeros_like(xcp))[0]
print(xinv, type(y), type(xinv))

[0.9999999 1.        1.        1.        1.       ] <class 'numpy.ndarray'> <class 'cupy.ndarray'>
[1.        1.        0.9999999 1.        1.       ] <class 'numpy.ndarray'> <class 'cupy.ndarray'>


## Blockdiag

In [7]:
nops = 5 # number of operators
n = 4

# Create operator
Ms = [np.diag((i + 1) * np.ones(n, dtype=dtype)) for i in range(nops)]
Ms = [M.T @ M for M in Ms]
M = block_diag(*Ms)
Mop = BlockDiag([MatrixMult(M, dtype=dtype) for M in Ms])

# Create data
x = np.ones(nops * n, dtype=dtype)
y = Mop @ x

# Invert
xinv = Mop / y
print(xinv)

[1.00000003 1.00000003 1.00000003 1.00000003 1.00000018 1.00000018
 1.00000018 1.00000018 0.99999999 0.99999999 0.99999999 0.99999999
 1.00000001 1.00000001 1.00000001 1.00000001 0.99999999 0.99999999
 0.99999999 0.99999999]


### Blockdiag with numpy input and numpy output

In [8]:
Mops = []
for iop in range(nops):
    Mop = MatrixMult(cp.asarray(Ms[iop], dtype=dtype))
    Top = ToCupy(Mop.dims, dtype=dtype)
    Top1 = ToCupy(Mop.dimsd, dtype=dtype)
    Mop = Top1.H @ Mop @ Top
    Mops.append(Mop)
Mops = BlockDiag(Mops, forceflat=True)

In [9]:
y = Mops * x.ravel()

xinv = pylops_cgls(Mops, y)[0]
print(xinv, type(y), type(xinv))

xinv = pylops_lsqr(Mops, y)[0]
print(xinv, type(y), type(xinv))

[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.] <class 'numpy.ndarray'> <class 'numpy.ndarray'>
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.] <class 'numpy.ndarray'> <class 'numpy.ndarray'>


### Blockdiag with cupy input and numpy output

In [10]:
Mops = []
for iop in range(nops):
    Mop = MatrixMult(cp.asarray(Ms[iop]), dtype=dtype)
    Top1 = ToCupy(Mop.dimsd, dtype=dtype)
    Mop = Top1.H @ Mop
    Mops.append(Mop)
Mops = BlockDiag(Mops, forceflat=True, inoutengine=("numpy", "cupy"))

In [11]:
xcp = cp.asarray(x)
y = Mops * xcp.ravel()

xinv = pylops_cg(Mops, y, x0=cp.zeros_like(xcp))[0]
print(xinv, type(y), type(xinv))

xinv = pylops_cgls(Mops, y, x0=cp.zeros_like(xcp))[0]
print(xinv, type(y), type(xinv))

xinv = pylops_lsqr(Mops, y, x0=cp.zeros_like(xcp))[0]
print(xinv, type(y), type(xinv))

[1.         1.         1.         1.         0.99999994 0.99999994
 0.99999994 0.99999994 1.0000001  1.0000001  1.0000001  1.0000001
 0.9999999  0.9999999  0.9999999  0.9999999  1.000001   1.000001
 1.000001   1.000001  ] <class 'numpy.ndarray'> <class 'cupy.ndarray'>
[0.99999994 0.99999994 0.99999994 0.99999994 1.0000011  1.0000011
 1.0000011  1.0000011  0.99999654 0.99999654 0.99999654 0.99999654
 1.         1.         1.         1.         1.0000001  1.0000001
 1.0000001  1.0000001 ] <class 'numpy.ndarray'> <class 'cupy.ndarray'>
[0.9999999  0.9999999  0.9999999  0.9999999  0.9999987  0.9999987
 0.9999987  0.9999987  0.9999997  0.9999997  0.9999997  0.9999997
 0.99999994 0.99999994 0.99999994 0.99999994 1.0000002  1.0000002
 1.0000002  1.0000002 ] <class 'numpy.ndarray'> <class 'cupy.ndarray'>


## HStack

In [12]:
nops = 5 # number of operators
n, m = 4, 5

# Create operator
Ms = [np.random.normal(0, 1, (n, m)) for _ in range(nops)]
M = np.hstack(Ms)
Mop = MatrixMult(M, dtype=dtype)

# Create data
x = np.ones(m * nops, dtype=dtype)
y = Mop @ x

# Invert
xinv = pylops_lsqr(Mop, y, niter=10)[0]
print(xinv)

[-0.02132629  0.36718273 -0.51855939 -0.08249748  0.58630573  0.28503575
  1.33158497  0.62250834  0.77536288  0.4021877  -0.02563346  0.47188322
  0.51167144  0.20568016  0.3829824  -0.67730089  0.26081406 -0.55516757
  0.94834444  0.65043393  0.39014753 -0.09388923  0.54119229  0.21238028
  0.13775378]


### HStack with numpy input and numpy output

In [13]:
Mops = []
for iop in range(nops):
    Mop = MatrixMult(cp.asarray(Ms[iop]), dtype=dtype)
    Top = ToCupy(Mop.dims, dtype=dtype)
    Top1 = ToCupy(Mop.dimsd, dtype=dtype)
    Mop = Top1.H @ Mop @ Top
    Mops.append(Mop)
Mops = HStack(Mops)

In [14]:
y = Mops * x.ravel()

xinv = pylops_cgls(Mops, y)[0]
print(xinv, type(y), type(xinv))

xinv = pylops_lsqr(Mops, y)[0]
print(xinv, type(y), type(xinv))

[-0.0213263   0.36718273 -0.5185594  -0.08249746  0.5863057   0.2850358
  1.3315849   0.6225083   0.7753629   0.4021877  -0.02563347  0.47188324
  0.51167136  0.20568009  0.38298237 -0.67730093  0.26081407 -0.5551676
  0.94834435  0.65043384  0.39014757 -0.09388922  0.5411923   0.21238033
  0.13775381] <class 'numpy.ndarray'> <class 'numpy.ndarray'>
[-0.02132629  0.36718273 -0.51855946 -0.08249752  0.58630573  0.28503569
  1.33158502  0.62250833  0.77536282  0.40218773 -0.02563342  0.47188321
  0.51167142  0.20568022  0.38298239 -0.6773009   0.26081407 -0.55516752
  0.94834444  0.6504339   0.39014756 -0.09388922  0.54119229  0.21238024
  0.13775377] <class 'numpy.ndarray'> <class 'numpy.ndarray'>


### HStack with cupy input and numpy output

In [15]:
Mops = []
for iop in range(nops):
    Mop = MatrixMult(cp.asarray(Ms[iop]), dtype=dtype)
    Top1 = ToCupy(Mop.dimsd, dtype=dtype)
    Mop = Top1.H @ Mop
    Mops.append(Mop)
Mops = HStack(Mops, inoutengine=("numpy", "cupy"))

In [16]:
xcp = cp.asarray(x)
y = Mops * xcp.ravel()

xinv = pylops_cgls(Mops, y, x0=cp.zeros_like(xcp))[0]
print(xinv, type(y), type(xinv))

xinv = pylops_lsqr(Mops, y, x0=cp.zeros_like(xcp))[0]
print(xinv, type(y), type(xinv))

[-0.0213263   0.36718273 -0.5185594  -0.08249746  0.5863057   0.2850358
  1.3315849   0.6225083   0.7753629   0.4021877  -0.02563347  0.47188324
  0.51167136  0.20568009  0.38298237 -0.67730093  0.26081407 -0.5551676
  0.94834435  0.65043384  0.39014757 -0.09388922  0.5411923   0.21238033
  0.13775381] <class 'numpy.ndarray'> <class 'cupy.ndarray'>
[-0.02132629  0.36718273 -0.5185595  -0.08249751  0.5863057   0.28503567
  1.331585    0.6225083   0.7753628   0.40218773 -0.02563342  0.47188318
  0.51167136  0.20568025  0.38298243 -0.6773009   0.26081407 -0.5551675
  0.9483444   0.65043396  0.39014754 -0.09388923  0.5411923   0.21238023
  0.13775375] <class 'numpy.ndarray'> <class 'cupy.ndarray'>
